<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch03/03_karaage_ai_book_bert_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTの日本語事前学習済みモデルをGoogle Colaboratoryで手軽に試す方法

## 必要なライブラリのセットアップ

transformersのインストール

In [1]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.1MB/s 
     |████████████████████████████████| 2.9MB 38.7MB/s 
     |████████████████████████████████| 890kB 40.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6fc0ae86576f288e4d2e816d25c70424994df97efff343d219e1dfc11bd98bb3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


transformersをソースからインストール（エラー対策のための一時的な処置です。`!pip install transformers`がうまくいく場合は、以下は不要です）

In [ ]:
#!git clone https://github.com/huggingface/transformers
#%cd transformers
#!pip install .

MeCabと関係ライブラリのインストール

https://qiita.com/pytry3g/items/897ae738b8fbd3ae7893

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

## BERTで単語の穴埋め

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
# Tokenize input
text = '父の父は、祖父。'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 2
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)

['父', 'の', '父', 'は', '、', '祖父', '。']
['父', 'の', '[MASK]', 'は', '、', '祖父', '。']
[800, 5, 4, 9, 6, 5235, 8]
tensor([[ 800,    5,    4,    9,    6, 5235,    8]])


In [ ]:
# Load pre-trained model
model_mask = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model_mask.eval()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
# Predict
with torch.no_grad():
    outputs = model_mask(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

0 の
1 、
2 と
3 父
4 母


## BERTで文章の特徴量抽出

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
model_bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model_bert.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

特徴量の計算。Bertの公式によると、あまりよくないかもとのことらしい

In [ ]:
import numpy as np

def calc_embedding(text):
  bert_tokens = tokenizer.tokenize(text)
  ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
  tokens_tensor = torch.tensor(ids).reshape(1, -1)

  with torch.no_grad():
    output = model_bert(tokens_tensor)

  return output[1].numpy()

In [ ]:
text = '私はからあげです'
calc_embedding(text)

array([[-4.80720028e-02,  4.65253592e-01, -1.83069155e-01,
        -3.77458304e-01,  9.60655436e-02,  9.98048246e-01,
        -7.86744058e-02, -1.10579789e-01, -2.16161519e-01,
         2.70328403e-01,  6.12610698e-01, -5.22115082e-02,
        -9.10325255e-03, -2.11077482e-01,  1.13648906e-01,
        -1.01962879e-01,  1.15811937e-02,  9.70593542e-02,
         2.84477741e-01,  1.56309366e-01,  9.21596587e-02,
         3.41767788e-01, -3.17929268e-01,  9.66841936e-01,
         9.99929190e-01,  1.15791239e-01,  4.14485961e-01,
        -5.67455292e-01,  9.99083221e-01,  8.55081439e-01,
        -7.46547997e-01,  4.08208877e-01, -8.66945684e-02,
         3.22805405e-01,  3.65310758e-01,  3.17443997e-01,
         1.38259321e-01, -2.39119515e-01, -3.83763134e-01,
         1.85939938e-01,  1.97876114e-02, -9.98861074e-01,
        -3.62229556e-01, -6.72740638e-02,  7.23658085e-01,
         1.12314291e-01,  4.84685391e-01, -2.40227252e-01,
         2.71576434e-01, -1.90293044e-01, -3.45357209e-0

# 参考リンク

- https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel
- https://yag-ays.github.io/project/pytorch_bert_japanese/
- https://dev.classmethod.jp/articles/bert-text-embedding/
- https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3
- https://qiita.com/yuki_uchida/items/09fda4c5c608a9f53d2f